# Moscow Marathon Full Results 2018


## 1_full_results_mm_2018.csv

Columns : {Bib, finish_time_sec, finish_time_result, race, pace_sec, pace(minpkm), pace(kmph), half_pace_sec, half_pace(minpkm), half_pace(kmph), gender_en, agev name_en, location_city_ru, location_city_en, country_code_alpha_3, flag_DNF, flag_all_split_exist, race_uniform_index}

Data : {1, 8911, 2h 28min 31sec, 42.195 km, 211.1861595, 3:31 min/km 17.0 km/h, 208.3185212, 3:28 min/km, 17.3 km/h, Female, 30, Sardana Trofimova, –Ø–∫—É—Ç—Å–∫, Yakutsk, RUS, 0, 1, 0.000132899}

## 1_split_results_mm_2018.csv

Columns : {bib, split_name, split, split_time_sec, split_time_result, split_pace_sec, split_pace(minpkm), split_pace(kmph), split_uniform_index}

Data : {11, Kirui, Geoffrey, 24, M, Keringet, KEN, 0:15:25, 0:30:28, 0:45:44, 1:01:15, 1:04:35, 1:16:59, 1:33:01, 1:48:19, 2:02:53, 0:04:57, - 2:09:37, 1, 1, 11}

In [47]:
from datetime import datetime
import numpy as np
import pandas as pd

# 데이터 불러오기
full_df = pd.read_csv('./data/1_full_results_mm_2018.csv')
split_df = pd.read_csv('./data/1_split_results_mm_2018.csv')

# bib 통일
full_df['bib'] = full_df['bib'].astype(str)
split_df['bib'] = split_df['bib'].astype(str)

# --------------------------
# split_time_sec pivot (5K~40K만)
split_time = split_df.pivot_table(index='bib', columns='split_name', values='split_time_sec')

# Marathon 열 제거
split_time = split_time.drop(columns=['Half marathon','Marathon'], errors='ignore')

# ' km' 제거 후 'K' 붙이기
split_time.columns = [col.replace(' km', '') + 'K' for col in split_time.columns]
split_time = split_time.apply(pd.to_numeric, errors='coerce')
split_time.reset_index(inplace=True)

# --------------------------
# split_pace_sec pivot (5p~40p만)
split_pace = split_df.pivot_table(index='bib', columns='split_name', values='split_pace_sec')
split_pace = split_pace.drop(columns=['Half marathon', 'Marathon'], errors='ignore')

split_pace.columns = [col.replace(' km', '') + 'p' for col in split_pace.columns]
split_pace = split_pace.apply(pd.to_numeric, errors='coerce')
split_pace.reset_index(inplace=True)

#--------------------------------------
# Half marathon 기록 추출
half_time = split_df[split_df['split_name'] == 'Half marathon'][['bib', 'split_time_sec']].copy()
half_time.rename(columns={'split_time_sec': 'Half'}, inplace=True)

half_pace = split_df[split_df['split_name'] == 'Half marathon'][['bib', 'split_pace_sec']].copy()
half_pace.rename(columns={'split_pace_sec': 'Halfp'}, inplace=True)

# 병합
split_time = split_time.merge(half_time, how='left', on='bib')
split_pace = split_pace.merge(half_pace, how='left', on='bib')



# --------------------------
# 필요한 컬럼 선택 및 전처리
reduced_df = full_df[['bib', 'age', 'gender_en', 'country_code_alpha_3', 'finish_time_sec']].copy()
reduced_df.rename(columns={
    'bib': 'Bib',
    'age': 'Age',
    'gender_en': 'M/F',
    'country_code_alpha_3': 'Country',
    'finish_time_sec': 'Final_Time'
}, inplace=True)

# Final_Time이 NaN인 경우 제거
reduced_df = reduced_df.dropna(subset=['Final_Time'])

# Age_group (19이하 → 19, ..., 70 이상 → 70)
def age_group(age):
    if age <= 19:
        return 19
    elif age >= 70:
        return 70
    else:
        return (age // 5) * 5 + 4  # 20~24 → 24, 25~29 → 29, ...

reduced_df['Age_group'] = reduced_df['Age'].apply(age_group)

# M/F: Male → 0, Female → 1
reduced_df['M/F'] = reduced_df['M/F'].map({'Male': 0, 'Female': 1})

# Sub 그룹핑
def set_sub_group(seconds):
    if pd.isna(seconds):
        return pd.NA
    hours = seconds / 3600
    if hours < 3:
        return 3
    elif hours < 4:
        return 4
    elif hours < 5:
        return 5
    elif hours < 6:
        return 6
    else:
        return 7

reduced_df['Sub'] = reduced_df['Final_Time'].apply(set_sub_group).astype('Int64')

# --------------------------
# 병합
merged = reduced_df.merge(split_time, how='left', left_on='Bib', right_on='bib')
merged = merged.merge(split_pace, how='left', on='bib')
merged.drop(columns=['bib'], inplace=True)

# Dataset 컬럼 추가
merged['Dataset'] = 'M'

merged['Year'] = 2018

merged.info()




# --------------------------
# 컬럼 순서 지정 (5~40K, 5~40p만 포함)
pace_cols = [f'{k}p' for k in range(5, 45, 5)]
time_cols = [f'{k}K' for k in range(5, 45, 5)]

columns_order = ['Bib', 'Age_group', 'M/F', 'Country'] + \
                pace_cols + ['Final_Time', 'Sub'] + \
                time_cols + ['Dataset'] + \
                ['Year', 'Half', 'Halfp']

# 존재하는 컬럼만 유지
columns_order = [col for col in columns_order if col in merged.columns]

# 결측치 제거
Moscow_df = merged[columns_order].dropna()

# Bib 재설정
Moscow_df['Bib'] = range(1, len(Moscow_df) + 1)
cols = Moscow_df.columns.tolist()
cols.remove('Bib')
Moscow_df = Moscow_df[['Bib'] + cols]

# int 변환
int_cols = Moscow_df.columns.difference(['Country', 'Dataset'])
Moscow_df[int_cols] = Moscow_df[int_cols].astype(int)

Moscow_df = Moscow_df[["Bib", "Age_group", "M/F", "Country", "5p", "10p", "15p", "Halfp", "25p", "30p", "35p", "40p", "Final_Time", "Sub", "5K", "10K", "15K", "Half", "25K", "30K", "35K", "40K", "Dataset", "Year"]]


# 저장
Moscow_df.to_csv('./data/Moscow_Marathon_Processed.csv', index=False)

# 확인
print(Moscow_df.head())
# Moscow_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21385 entries, 0 to 21384
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Bib         21385 non-null  object 
 1   Age         21385 non-null  int64  
 2   M/F         21385 non-null  int64  
 3   Country     21385 non-null  object 
 4   Final_Time  21385 non-null  float64
 5   Age_group   21385 non-null  int64  
 6   Sub         21385 non-null  Int64  
 7   10K         21384 non-null  float64
 8   15K         8692 non-null   float64
 9   25K         8692 non-null   float64
 10  30K         8670 non-null   float64
 11  35K         8659 non-null   float64
 12  40K         8656 non-null   float64
 13  5K          21361 non-null  float64
 14  Half        8702 non-null   float64
 15  10p         21384 non-null  float64
 16  15p         8692 non-null   float64
 17  25p         8692 non-null   float64
 18  30p         8670 non-null   float64
 19  35p         8659 non-null

# Finishers Boston Marathon 2015, 2016 & 2017

In [48]:
import pandas as pd
import numpy as np

# CSV 파일 로드
df_15 = pd.read_csv('./data/marathon_results_2015.csv')
df_16 = pd.read_csv('./data/marathon_results_2016.csv')
df_17 = pd.read_csv('./data/marathon_results_2017.csv')

# 연도 컬럼 추가
df_15['Year'] = 2015
df_16['Year'] = 2016
df_17['Year'] = 2017

# 데이터 통합
df = pd.concat([df_15, df_16, df_17], ignore_index=True)

df = df.rename(columns={"Half": "HalfK"})


# 불필요한 컬럼 제거
drop_cols = ['Unnamed: 0', 'Unnamed: 8', 'Unnamed: 9', 'State', 'Citizen', 'Proj Time']
df = df.drop(columns=[col for col in drop_cols if col in df.columns])

# 시간형 컬럼 처리 대상: 21K는 제거 대상
time_cols = ['5K', '10K', '15K', 'HalfK', '25K', '30K', '35K', '40K', 'Pace', 'Official Time']
for col in time_cols:
    df[col] = pd.to_timedelta(df[col], errors='coerce')

# 초 단위로 변환
for col in ['5K', '10K', '15K', 'HalfK', '25K', '30K', '35K', '40K', 'Official Time']:
    df[col] = df[col].dt.total_seconds()

# 컬럼명 변경
df.rename(columns={'Official Time': 'Final_Time'}, inplace=True)

# 페이스 계산 (21K 제외)
distance_km = {'5K': 5, '10K': 10, '15K': 15, 'HalfK': 21.0975, '25K': 25, '30K': 30, '35K': 35, '40K': 40}
for dist, km in distance_km.items():
    pace_col = dist.replace('K', 'p')
    df[pace_col] = df[dist] / km

# Age_group 지정
def age_group(age):
    if age < 20:
        return 19
    elif age < 25:
        return 24
    elif age < 30:
        return 29
    elif age < 35:
        return 34
    elif age < 40:
        return 39
    elif age < 45:
        return 44
    elif age < 50:
        return 49
    elif age < 55:
        return 54
    elif age < 60:
        return 59
    elif age < 65:
        return 64
    elif age < 70:
        return 69
    else:
        return 70

df['Age_group'] = df['Age'].apply(age_group)

# 성별 인코딩
df['M/F'] = df['M/F'].map({'M': 0, 'F': 1})

# Sub (시간 그룹)
def sub_group(time_sec):
    hours = time_sec / 3600
    if hours <= 3:
        return 3
    elif hours <= 4:
        return 4
    elif hours <= 5:
        return 5
    elif hours <= 6:
        return 6
    else:
        return 7

df['Sub'] = df['Final_Time'].apply(sub_group)

# 필요한 컬럼만 추출
base_cols = ['Bib', 'Age_group', 'M/F', 'Country']
pace_cols = [k.replace('K', 'p') for k in distance_km.keys()]
time_cols = list(distance_km.keys())
final_cols = ['Final_Time', 'Sub']
df['Dataset'] = 'B'

ordered_cols = base_cols + pace_cols + final_cols + time_cols + ['Dataset', 'Year']
df = df[ordered_cols]

# 결측치 제거 및 Bib 재할당
df = df.dropna()
df.reset_index(drop=True, inplace=True)
df['Bib'] = df.index + 1

# 숫자형 컬럼 int로 변환
int_cols = df.columns.difference(['Country', 'Dataset'])
df[int_cols] = df[int_cols].astype(int)

df = df.rename(columns={"HalfK": "Half"})


# 저장
df.to_csv('./data/boston_processed.csv', index=False)

# 결과 확인
print(df.head())
# df.info()

   Bib  Age_group  M/F Country   5p  10p  15p  Halfp  25p  30p  ...   5K  \
0    1         29    0     ETH  176  178  179    182  182  184  ...  883   
1    2         34    0     ETH  176  178  179    182  182  183  ...  883   
2    3         29    0     KEN  176  178  179    182  182  184  ...  883   
3    4         29    0     KEN  176  178  180    182  182  184  ...  883   
4    5         34    0     KEN  176  178  179    182  182  184  ...  883   

    10K   15K  Half   25K   30K   35K   40K  Dataset  Year  
0  1783  2697  3842  4567  5520  6479  7359        B  2015  
1  1783  2698  3841  4567  5519  6479  7362        B  2015  
2  1783  2697  3842  4567  5520  6479  7381        B  2015  
3  1784  2701  3842  4567  5520  6483  7427        B  2015  
4  1784  2698  3841  4567  5520  6479  7407        B  2015  

[5 rows x 24 columns]


In [49]:
import pandas as pd

# 파일 경로
boston_path = './data/boston_processed.csv'
moscow_path = './data/Moscow_Marathon_Processed.csv'

# 데이터 불러오기
df_boston = pd.read_csv(boston_path)
df_moscow = pd.read_csv(moscow_path)

# 병합 (인덱스 초기화)
df_merged = pd.concat([df_boston, df_moscow], ignore_index=True)

# Bib 재설정 (1부터 시작)
df_merged['Bib'] = range(1, len(df_merged) + 1)

# Bib을 맨 앞으로 이동
cols = df_merged.columns.tolist()
cols.remove('Bib')
df_merged = df_merged[['Bib'] + cols]

# 결측치 확인 (추가적인 안전 확인)
print("결측치 존재 여부:\n", df_merged.isnull().sum().sum())  # 0이면 OK

# Dataset별 샘플 수 확인
print("Dataset 분포:\n", df_merged['Dataset'].value_counts())

# 저장
df_merged.to_csv('./data/combined_Marathon_Data.csv', index=False)
print(f"✔️ 병합된 데이터 저장 완료! 총 샘플 수: {len(df_merged)}")

결측치 존재 여부:
 0
Dataset 분포:
 Dataset
B    79073
M     8611
Name: count, dtype: int64
✔️ 병합된 데이터 저장 완료! 총 샘플 수: 87684


In [50]:
# import pandas as pd

# # 파일 경로
# boston_path = './data/boston_processed.csv'
# moscow_path = './data/Moscow_Marathon_Processed.csv'

# # 데이터 불러오기
# df_boston = pd.read_csv(boston_path)
# df_moscow = pd.read_csv(moscow_path)

# # 데이터 병합
# df_merged = pd.concat([df_boston, df_moscow], ignore_index=True)

# # 결측치 제거
# df_merged.dropna(inplace=True)

# # 정수형 변환 (Country, Dataset 제외)
# exclude_cols = ['Country', 'Dataset']
# int_cols = df_merged.columns.difference(exclude_cols)
# df_merged[int_cols] = df_merged[int_cols].astype(int)

# # Bib 재설정 및 정렬
# df_merged['Bib'] = range(1, len(df_merged) + 1)
# cols = df_merged.columns.tolist()
# cols.remove('Bib')
# df_merged = df_merged[['Bib'] + cols]

# # 저장
# df_merged.to_csv('./data/combined_Marathon_Data.csv', index=False)

# print("병합된 데이터 샘플 수:", len(df_merged))
# print("결측치 확인:", df_merged.isnull().sum().sum())

In [51]:
import pandas as pd

# 각 데이터셋 불러오기
combined_df = pd.read_csv("./data/Combined_Marathon_Data.csv")      # 보스턴 등 기존 데이터
chicago_df = pd.read_csv("./data/chicago_data_processed.csv")       # 시카고 데이터

# 두 데이터셋을 하나로 병합 (인덱스 재정렬 포함)
merged_df = pd.concat([combined_df, chicago_df], ignore_index=True)

# (선택) 저장하고 싶다면
merged_df.to_csv("./data/merged_marathon_data.csv", index=False)


In [52]:
len(df_moscow)

8611

In [53]:
import numpy as np

# 수치형 컬럼만 선택
numeric_df = df_merged.select_dtypes(include=[np.number])

# NaN 또는 inf 값이 있는 행의 마스크
non_finite_mask = ~np.isfinite(numeric_df)

# 마스크로 해당 행 추출
rows_with_nan_or_inf = df_merged[non_finite_mask.any(axis=1)]

# 결과 출력
print(f"NaN 또는 inf 값을 포함한 행 수: {len(rows_with_nan_or_inf)}")
display(rows_with_nan_or_inf)

NaN 또는 inf 값을 포함한 행 수: 0


,Bib,Age_group,M/F,Country,5p,10p,15p,Halfp,25p,30p,...,5K,10K,15K,Half,25K,30K,35K,40K,Dataset,Year


In [54]:
count_M = (merged_df['Dataset'] == 'M').sum()
count_B = (merged_df['Dataset'] == 'B').sum()
count_C = (merged_df['Dataset'] == 'C').sum()

print(f"Dataset = 'M'인 행 수: {count_M}")
print(f"Dataset = 'B'인 행 수: {count_B}")
print(f"Dataset = 'C'인 행 수: {count_C}")

Dataset = 'M'인 행 수: 8611
Dataset = 'B'인 행 수: 79073
Dataset = 'C'인 행 수: 26928


In [55]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114612 entries, 0 to 114611
Data columns (total 24 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Bib         114612 non-null  int64  
 1   Age_group   114612 non-null  int64  
 2   M/F         114612 non-null  int64  
 3   Country     114612 non-null  object 
 4   5p          114489 non-null  float64
 5   10p         114596 non-null  float64
 6   15p         114600 non-null  float64
 7   Halfp       114604 non-null  float64
 8   25p         114605 non-null  float64
 9   30p         114606 non-null  float64
 10  35p         114607 non-null  float64
 11  40p         114607 non-null  float64
 12  Final_Time  114612 non-null  float64
 13  Sub         114612 non-null  int64  
 14  5K          114489 non-null  float64
 15  10K         114596 non-null  float64
 16  15K         114600 non-null  float64
 17  Half        114604 non-null  float64
 18  25K         114605 non-null  float64
 19  30

In [57]:
merged_df.head()

,Bib,Age_group,M/F,Country,5p,10p,15p,Halfp,25p,30p,...,5K,10K,15K,Half,25K,30K,35K,40K,Dataset,Year
0,1,29,0,ETH,176.0,178.0,179.0,182.0,182.0,184.0,...,883.0,1783.0,2697.0,3842.0,4567.0,5520.0,6479.0,7359.0,B,2015
1,2,34,0,ETH,176.0,178.0,179.0,182.0,182.0,183.0,...,883.0,1783.0,2698.0,3841.0,4567.0,5519.0,6479.0,7362.0,B,2015
2,3,29,0,KEN,176.0,178.0,179.0,182.0,182.0,184.0,...,883.0,1783.0,2697.0,3842.0,4567.0,5520.0,6479.0,7381.0,B,2015
3,4,29,0,KEN,176.0,178.0,180.0,182.0,182.0,184.0,...,883.0,1784.0,2701.0,3842.0,4567.0,5520.0,6483.0,7427.0,B,2015
4,5,34,0,KEN,176.0,178.0,179.0,182.0,182.0,184.0,...,883.0,1784.0,2698.0,3841.0,4567.0,5520.0,6479.0,7407.0,B,2015


In [56]:

# 20p가 결측인 데이터만 추출
df_missing_20p = merged_df[merged_df["20p"].isna()]

# Dataset 컬럼의 값 개수 세기
dataset_counts = df_missing_20p["Dataset"].value_counts()

# 결과 출력
print(dataset_counts)


KeyError: '20p'

In [ ]:
# import numpy as np

# # NaN 또는 inf 값이 있는 행을 찾기 위한 마스크 생성
# non_finite_mask = ~np.isfinite(df_merged)

# # 마스크를 사용하여 해당 행만 추출
# rows_with_nan_or_inf = df_merged[non_finite_mask.any(axis=1)]

# # 결과 출력
# print(f"NaN 또는 inf가 포함된 행 수: {len(rows_with_nan_or_inf)}")
# display(rows_with_nan_or_inf)